In [2]:
%load_ext lab_black

In [1]:
import solike.clusters

In [4]:
import camb

In [3]:
import numpy as np
%pylab inline
import matplotlib.pyplot as plt
import time

fiducial_params = {
    'ombh2': 0.02225, 'omch2': 0.1198, 'H0': 67.3, 'tau': 0.06,
    'As': 2.2e-9, 'ns': 0.96,
    'mnu': 0.06, 'nnu': 3.046, 'num_massive_neutrinos': 1}
l_max = 1000
zarr = np.linspace(0,2,41)
modules_path = '/Users/nab/Repos/cobaya_modules/'

def pk(_theory={'Pk_interpolator': {'z': np.linspace(0,4,41), 'k_max': 100.0, 'nonlinear': True,'hubble_units': True,'k_hunit': True, 'vars_pairs': [['delta_nonu','delta_nonu']]}}):
    #print (_theory.get_Pk_interpolator())
    Pk_interpolator = _theory.get_Pk_interpolator()['delta_nonu_delta_nonu'].P
    k = np.logspace(-3,1,100)
    return Pk_interpolator(0.0,k)

info_fiducial = {
    'params': fiducial_params,
    'likelihood': {'pks': pk},
    'theory': {'camb':{'extra_args':{"accurate_massive_neutrino_transfers": True,
                                                   "redshifts": np.linspace(0,2,41), "nonlinear": False,
                                                   "kmax": 10., "dark_energy_model":"ppf"}}
              },
    'modules': modules_path}

from cobaya.model import get_model
model_fiducial = get_model(info_fiducial)
print ('loglike', model_fiducial.loglike({'As': 3e-9, 'ns':0.96}))
# Declare our desired theory product
# (there is no cosmological likelihood doing it for us)
model_fiducial.likelihood.theory.needs(Cl={'tt': l_max},
                                       Pk_interpolator={'z': np.linspace(0,2,41), 'k_max': 5.0,
                                                        'nonlinear': False,'hubble_units': True,'k_hunit': True,
                                                        'vars_pairs': [['delta_tot','delta_tot']]},
                                       H={'z': np.linspace(0,2,41)})#,
                                       #extra_args={"accurate_massive_neutrino_transfers": True,
                                       #            "redshifts": np.linspace(0,4,41), "nonlinear": False,
                                       #            "kmax": 10., "dark_energy_model":"ppf"})
# Compute and extract the CMB power spectrum
# (In muK^-2, without l(l+1)/(2pi) factor)
# notice the empty dictionary below: all parameters are fixed
model_fiducial.logposterior({})
Cls = model_fiducial.likelihood.theory.get_Cl(ell_factor=False)
print(model_fiducial.likelihood.theory.requested())
Pk_interpolator = model_fiducial.likelihood.theory.get_Pk_interpolator()['delta_nonu_delta_nonu'].P#(0,k_max=5)
k = np.logspace(-4,np.log10(5),200)
pks = Pk_interpolator(zarr,k)
Ez = model_fiducial.likelihood.theory.get_H(zarr) / model_fiducial.likelihood.theory.get_param('H0')
om = (model_fiducial.likelihood.theory.get_param('omch2') + model_fiducial.likelihood.theory.get_param('ombh2'))/ ((model_fiducial.likelihood.theory.get_param('H0')/100.)**2)
print (pks.shape,Ez.shape)
import solike.clusters.massfunc as mf
hmf = mf.HMF(om,Ez,pk=pks,kh=k,zarr=zarr)

Populating the interactive namespace from numpy and matplotlib
[prior] *WARNING* No sampled parameters requested! This will fail for non-mock samplers.
[camb] *local* CAMB not found at /Users/nab/Repos/cobaya_modules/code/CAMB
[camb] Importing *global* CAMB.
[pks] Initialized external likelihood.
[camb] *ERROR* hubble_units and k_hunit must be Falsefor consistency


LoggedError: hubble_units and k_hunit must be Falsefor consistency

In [1]:
from solike import ClusterLikelihood

import numpy as np

fiducial_params = {
    'ombh2': 0.02225, 'omch2': 0.1198, 'H0': 67.3, 'tau': 0.06,
    'As': 2.2e-9, 'ns': 0.96,
    'mnu': 0.06, 'nnu': 3.046, 'num_massive_neutrinos': 1}

info_fiducial = {
    'params': fiducial_params,
    'likelihood': {'solike.ClusterLikelihood' : {'stop_at_error': True}},
    'theory': {'camb':{'extra_args':{"accurate_massive_neutrino_transfers": True,
                                                   "redshifts": np.linspace(0,2,41), "nonlinear": False,
                                                   "kmax": 10., "dark_energy_model":"ppf"}}
              }}

from cobaya.model import get_model
model_fiducial = get_model(info_fiducial)

[prior] *WARNING* No sampled parameters requested! This will fail for non-mock samplers.
[camb] Importing *global* CAMB.
mock catalog


In [2]:
like = model_fiducial.likelihood['solike.ClusterLikelihood']

In [3]:
model_fiducial.loglikes({})[0]

(350,)
(136,)
(136,)
[solike.clusterlikelihood] *ERROR* Error at evaluation: ValueError('operands could not be broadcast together with shapes (350,) (136,) ')


LoggedError: Error at evaluation: ValueError('operands could not be broadcast together with shapes (350,) (136,) ')

In [3]:
model_fiducial.loglikes({})[0]

(110, 1, 350)
[solike.clusterlikelihood] *ERROR* Error at evaluation: KeyError('Q')


LoggedError: Error at evaluation: KeyError('Q')

In [3]:
like = model_fiducial.likelihood['solike.ClusterLikelihood']

In [4]:
like._get_HMF()

AttributeError: 'camb' object has no attribute '_current_state'

In [5]:
surveydata = like._get_catalog()

mock catalog


In [ ]:
surveydata.

In [3]:


def _get_catalog():
    catalog = Survey.SurveyData(self.data_path, self.data_name)
    return catalog

In [ ]:
def Prob_per_cluster(self, HMF, cluster_props, param_vals):
    # c_z, c_zerr, c_y, c_yerr = cluster_props
    tempz = cluster_props[0, :]
    zind = np.argsort(tempz)
    tempz = 0.
    c_z = cluster_props[0, zind]
    c_zerr = cluster_props[1, zind]
    c_y = cluster_props[2, zind]
    c_yerr = cluster_props[3, zind]

    Marr = np.outer(int_HMF.M.copy(), np.ones([len(c_z)]))
    zarr = np.outer(np.ones([len(int_HMF.M.copy())]), c_z)

    if (c_zerr.any() > 0):
        # FIX THIS
        z_arr = np.arange(-3.*c_zerr, (3.+0.1)*c_zerr, c_zerr) + c_z
        Pfunc_ind = self.Pfunc_per_zarr(int_HMF.M.copy(), z_arr, c_y, c_yerr, int_HMF, param_vals)
        M200 = int_HMF.cc.Mass_con_del_2_del_mean200(int_HMF.M.copy(), 500, c_z)  # FIX THIS?
        dn_dzdm = dn_dzdm_int(z_arr, np.log10(int_HMF.M.copy()))
        N_z_ind = np.trapz(dn_dzdm*Pfunc_ind, dx=np.diff(M200, axis=0), axis=0)
        N_per = np.trapz(N_z_ind*gaussian(z_arr, c_z, c_zerr), dx=np.diff(z_arr))
        ans = N_per
    else:
        Pfunc_ind = self.Pfunc_per(Marr, zarr, c_y, c_yerr, param_vals)
        dn_dzdm = HMF.dn_dzdm(c_z, np.log10(int_HMF.M.copy()))
        M200 = int_HMF.M200_int(c_z, int_HMF.M.copy())
        N_z_ind = np.trapz(dn_dzdm*Pfunc_ind, dx=np.diff(M200, axis=0), axis=0)
        ans = N_z_ind
    return ans


In [ ]:
def Prob_per_cluster(HMF, z, y, param_vals):
    

In [1]:
from pkg_resources import resource_filename

resource_filename('solike.clusters', 'data/ACTPol_Cond_scatv5.fits')

'/Users/tmorton/repositories/likelihoods/solike/clusters/data/ACTPol_Cond_scatv5.fits'